In [3]:

import os
from tqdm import tqdm
import cv2
import torch
import numpy as np

In [4]:
gt_path = '../testdata/labels/'
preds_path = '../testdata/preds/'

# get all imgs'name, remove the extension
images_folder1 = [f for f in os.listdir(gt_path) if
                  os.path.isfile(os.path.join(gt_path, f)) and f.lower().endswith(
                      ('.png', '.jpg', '.jpeg', '.bmp', '.tiff', '.gif'))]
images_names = [os.path.splitext(f)[0] for f in images_folder1]
tbar = tqdm(images_names)



  0%|          | 0/21 [00:03<?, ?it/s]


# Target Center Metric

In [5]:
from sosmetrics.metrics import TargetPrecisionRecallF1
# For test multiple format of input.
gt_img_list = []
pred_img_list = [] 

gt_img_list_chw = []
pred_img_list_chw = []

gt_img_paths = []
pred_img_paths = []

Metric = TargetPrecisionRecallF1(
    dis_thr=[1, 10],
    conf_thr=0.5,
    match_alg='forloop',
    # debug = True
    )
for image_name in tbar:
    tbar.set_description(f"Reading image_name={image_name}")
    gt_image_path = os.path.join(gt_path, f"{image_name}.png")  # 假设所有图片都是PNG格式，或者你可以根据实际情况修改
    pred_image_path = os.path.join(preds_path, f"{image_name}.png") 

    gt_img = cv2.imread(gt_image_path)
    pred_img = cv2.imread(pred_image_path)

    # for test [chw, chw, ...] format
    gt_img_list_chw.append(gt_img.transpose(2,1,0))
    pred_img_list_chw.append(pred_img.transpose(2,1,0))

    # for test [hwc, hwc] format
    gt_img_list.append(gt_img)
    pred_img_list.append(pred_img)

    # for test [path, path] format

    gt_img_paths.append(gt_image_path)
    pred_img_paths.append(pred_image_path)

    # for test single img or img path
    Metric.update(gt_img, pred_img)
    Metric.update(gt_image_path, pred_image_path)

print("Simultaneous test single image path and single image, (TP, FN, FP) should be two times as big as the following test.")
Metric.get()
Metric.reset()


print("Test image of list [hwc, hwc, ...]")
Metric.update(gt_img_list, pred_img_list) 
Metric.get()
Metric.reset()

print("Test image_path of list, [img_path, img_path, ...]")
Metric.update(gt_img_paths, pred_img_paths)
Metric.get()
Metric.reset()

print("Test image of np.array, bhwc")
Metric.update(np.stack(gt_img_list), np.stack(pred_img_list)) 
Metric.get()
Metric.reset()


print("Test image of tensor, bchw")
Metric.update(torch.from_numpy(np.stack(gt_img_list_chw)), torch.from_numpy(np.stack(pred_img_list_chw))) 
Metric.get()
Metric.reset()

print("Test image of list,  [chw, chw, ...]")
Metric.update(gt_img_list_chw, pred_img_list_chw) 
Metric.get()

Reading image_name=0020: 100%|██████████| 21/21 [00:03<00:00,  6.39it/s]


Simultaneous test single image path and single image, (TP, FN, FP) should be two times as big as the following test.
TargetPrecisionRecallF1.update() took 0.00s each time.
+---------+----+----+----+------------------+---------------+----------------+
| Dis-Thr | TP | FP | FN | target_Precision | target_Recall | target_F1score |
+---------+----+----+----+------------------+---------------+----------------+
|   1.0   | 38 | 20 | 14 |     0.65517      |    0.73077    |    0.69091     |
|   2.0   | 44 | 14 | 8  |     0.75862      |    0.84615    |    0.80000     |
|   3.0   | 44 | 14 | 8  |     0.75862      |    0.84615    |    0.80000     |
|   4.0   | 44 | 14 | 8  |     0.75862      |    0.84615    |    0.80000     |
|   5.0   | 44 | 14 | 8  |     0.75862      |    0.84615    |    0.80000     |
|   6.0   | 44 | 14 | 8  |     0.75862      |    0.84615    |    0.80000     |
|   7.0   | 44 | 14 | 8  |     0.75862      |    0.84615    |    0.80000     |
|   8.0   | 44 | 14 | 8  |     0.75862

(array([0.65517241, 0.75862069, 0.75862069, 0.75862069, 0.75862069,
        0.75862069, 0.75862069, 0.75862069, 0.75862069, 0.75862069]),
 array([0.73076923, 0.84615385, 0.84615385, 0.84615385, 0.84615385,
        0.84615385, 0.84615385, 0.84615385, 0.84615385, 0.84615385]),
 array([0.69090909, 0.8       , 0.8       , 0.8       , 0.8       ,
        0.8       , 0.8       , 0.8       , 0.8       , 0.8       ]))

In [6]:
Metric.table

,dis_thr,TP,FP,FN,Precision,Recall,F1
0,1.0,19.0,10.0,7.0,0.655172,0.730769,0.690909
1,2.0,22.0,7.0,4.0,0.758621,0.846154,0.800000
2,3.0,22.0,7.0,4.0,0.758621,0.846154,0.800000
3,4.0,22.0,7.0,4.0,0.758621,0.846154,0.800000
4,5.0,22.0,7.0,4.0,0.758621,0.846154,0.800000
5,6.0,22.0,7.0,4.0,0.758621,0.846154,0.800000
6,7.0,22.0,7.0,4.0,0.758621,0.846154,0.800000
7,8.0,22.0,7.0,4.0,0.758621,0.846154,0.800000
8,9.0,22.0,7.0,4.0,0.758621,0.846154,0.800000
9,10.0,22.0,7.0,4.0,0.758621,0.846154,0.800000


## Target Average Precision


In [7]:
from sosmetrics.metrics import TargetAveragePrecision
Metric = TargetAveragePrecision(
    dis_thr=[1, 10],
    conf_thr=10,
    match_alg='forloop',
    # debug = True
    )
# For test multiple format of input.
gt_img_list = []
pred_img_list = [] 

gt_img_list_chw = []
pred_img_list_chw = []

gt_img_paths = []
pred_img_paths = []
for image_name in tbar:
    tbar.set_description(f"Reading image_name={image_name}")
    gt_image_path = os.path.join(gt_path, f"{image_name}.png")  # 假设所有图片都是PNG格式，或者你可以根据实际情况修改
    pred_image_path = os.path.join(preds_path, f"{image_name}.png") 

    gt_img = cv2.imread(gt_image_path)
    pred_img = cv2.imread(pred_image_path)

    # for test [chw, chw, ...] format
    gt_img_list_chw.append(gt_img.transpose(2,1,0))
    pred_img_list_chw.append(pred_img.transpose(2,1,0))

    # for test [hwc, hwc] format
    gt_img_list.append(gt_img)
    pred_img_list.append(pred_img)

    # for test [path, path] format

    gt_img_paths.append(gt_image_path)
    pred_img_paths.append(pred_image_path)

    # for test single img or img path
    Metric.update(gt_img, pred_img)
    Metric.update(gt_image_path, pred_image_path)

print("Simultaneous test single image path and single image, (TP, FN, FP) should be two times as big as the following test.")
Metric.get()
Metric.reset()


print("Test image of list [hwc, hwc, ...]")
Metric.update(gt_img_list, pred_img_list) 
Metric.get()
Metric.reset()

print("Test image_path of list, [img_path, img_path, ...]")
Metric.update(gt_img_paths, pred_img_paths)
Metric.get()
Metric.reset()

print("Test image of np.array, bhwc")
Metric.update(np.stack(gt_img_list), np.stack(pred_img_list)) 
Metric.get()
Metric.reset()


print("Test image of tensor, bchw")
Metric.update(torch.from_numpy(np.stack(gt_img_list_chw)), torch.from_numpy(np.stack(pred_img_list_chw))) 
Metric.get()
Metric.reset()

print("Test image of list,  [chw, chw, ...]")
Metric.update(gt_img_list_chw, pred_img_list_chw) 
Metric.get()


Simultaneous test single image path and single image, (TP, FN, FP) should be two times as big as the following test.
TargetAveragePrecision.update() took 0.02s each time.
+---------+--------+--------+--------+--------+--------+--------+--------+--------+--------+--------+
| Dis—Thr |  1.0   |  2.0   |  3.0   |  4.0   |  5.0   |  6.0   |  7.0   |  8.0   |  9.0   |  10.0  |
+---------+--------+--------+--------+--------+--------+--------+--------+--------+--------+--------+
|    AP   | 0.4539 | 0.5236 | 0.5467 | 0.5689 | 0.5689 | 0.5689 | 0.5689 | 0.5689 | 0.5689 | 0.5689 |
+---------+--------+--------+--------+--------+--------+--------+--------+--------+--------+--------+
TargetAveragePrecision(match_alg=forloop)
Test image of list [hwc, hwc, ...]
TargetAveragePrecision.update() took 0.03s each time.
+---------+--------+--------+--------+--------+--------+--------+--------+--------+--------+--------+
| Dis—Thr |  1.0   |  2.0   |  3.0   |  4.0   |  5.0   |  6.0   |  7.0   |  8.0   |  9

(array([[0.        , 0.7       , 0.7       , 0.72413793, 0.72413793,
         0.63333333, 0.6       , 0.6       , 0.5862069 , 0.53333333],
        [0.        , 0.73333333, 0.73333333, 0.75862069, 0.75862069,
         0.73333333, 0.7       , 0.7       , 0.68965517, 0.6       ],
        [0.        , 0.73333333, 0.73333333, 0.75862069, 0.75862069,
         0.73333333, 0.73333333, 0.73333333, 0.72413793, 0.63333333],
        [0.        , 0.73333333, 0.73333333, 0.75862069, 0.75862069,
         0.73333333, 0.73333333, 0.73333333, 0.72413793, 0.66666667],
        [0.        , 0.73333333, 0.73333333, 0.75862069, 0.75862069,
         0.73333333, 0.73333333, 0.73333333, 0.72413793, 0.66666667],
        [0.        , 0.73333333, 0.73333333, 0.75862069, 0.75862069,
         0.73333333, 0.73333333, 0.73333333, 0.72413793, 0.66666667],
        [0.        , 0.73333333, 0.73333333, 0.75862069, 0.75862069,
         0.73333333, 0.73333333, 0.73333333, 0.72413793, 0.66666667],
        [0.        , 0.7333

In [8]:
Metric.table

,0,1,2,3,4,5,6,7,8,9
Dis-Thr,1.00000,2.000000,3.000000,4.000000,5.000000,6.000000,7.000000,8.000000,9.000000,10.000000
AP,0.45389,0.523563,0.546729,0.568877,0.568877,0.568877,0.568877,0.568877,0.568877,0.568877


# PD_FA Metric

In [9]:
# For test multiple format of input.
from sosmetrics.metrics import TargetPdPixelFa


gt_img_list = []
pred_img_list = [] 

gt_img_list_chw = []
pred_img_list_chw = []

gt_img_paths = []
pred_img_paths = []

Metric = TargetPdPixelFa(
    dis_thr=[1, 10],
    conf_thr=0.5,
    match_alg='forloop',
    # debug = True
    )
for image_name in tbar:
    tbar.set_description(f"Reading image_name={image_name}")
    gt_image_path = os.path.join(gt_path, f"{image_name}.png") 
    pred_image_path = os.path.join(preds_path, f"{image_name}.png") 

    gt_img = cv2.imread(gt_image_path)
    pred_img = cv2.imread(pred_image_path)

    # for test [chw, chw, ...] format
    gt_img_list_chw.append(gt_img.transpose(2,1,0))
    pred_img_list_chw.append(pred_img.transpose(2,1,0))

    # for test [hwc, hwc] format
    gt_img_list.append(gt_img)
    pred_img_list.append(pred_img)

    # for test [path, path] format
    gt_img_paths.append(gt_image_path)
    pred_img_paths.append(pred_image_path)

    # for test single img or img path
    Metric.update(gt_img, pred_img)
    Metric.update(gt_image_path, pred_image_path)

print("Simultaneous test single image path and single image, (TP, FN, FP) should be two times as big as the following test.")
Metric.get()
Metric.reset()


print("Test image of list [hwc, hwc, ...]")
Metric.update(gt_img_list, pred_img_list) 
Metric.get()
Metric.reset()

print("Test image_path of list, [img_path, img_path, ...]")
Metric.update(gt_img_paths, pred_img_paths)
Metric.get()
Metric.reset()

print("Test image of np.array, bhwc")
Metric.update(np.stack(gt_img_list), np.stack(pred_img_list)) 
Metric.get()
Metric.reset()


print("Test image of tensor, bchw")
Metric.update(torch.from_numpy(np.stack(gt_img_list_chw)), torch.from_numpy(np.stack(pred_img_list_chw))) 
Metric.get()
Metric.reset()

print("Test image of list,  [chw, chw, ...]")
Metric.update(gt_img_list_chw, pred_img_list_chw) 
Metric.get()


Simultaneous test single image path and single image, (TP, FN, FP) should be two times as big as the following test.
TargetPdPixelFa.update() took 0.01s each time.
+-----------+----+----+-----+---------+-----------+-------------+
| Threshold | TD | AT |  FD |    NP   | target_Pd |   pixel_Fa  |
+-----------+----+----+-----+---------+-----------+-------------+
|    1.0    | 38 | 52 | 300 | 2752512 |  0.73077  | 1.08991e-04 |
|    2.0    | 44 | 52 | 108 | 2752512 |  0.84615  | 3.92369e-05 |
|    3.0    | 44 | 52 | 108 | 2752512 |  0.84615  | 3.92369e-05 |
|    4.0    | 44 | 52 | 108 | 2752512 |  0.84615  | 3.92369e-05 |
|    5.0    | 44 | 52 | 108 | 2752512 |  0.84615  | 3.92369e-05 |
|    6.0    | 44 | 52 | 108 | 2752512 |  0.84615  | 3.92369e-05 |
|    7.0    | 44 | 52 | 108 | 2752512 |  0.84615  | 3.92369e-05 |
|    8.0    | 44 | 52 | 108 | 2752512 |  0.84615  | 3.92369e-05 |
|    9.0    | 44 | 52 | 108 | 2752512 |  0.84615  | 3.92369e-05 |
|    10.0   | 44 | 52 | 108 | 2752512 |  0.8

(array([0.73076923, 0.84615385, 0.84615385, 0.84615385, 0.84615385,
        0.84615385, 0.84615385, 0.84615385, 0.84615385, 0.84615385]),
 array([1.08991350e-04, 3.92368862e-05, 3.92368862e-05, 3.92368862e-05,
        3.92368862e-05, 3.92368862e-05, 3.92368862e-05, 3.92368862e-05,
        3.92368862e-05, 3.92368862e-05]))

In [10]:
Metric.table

,dis_thr,TD,AT,FD,NP,target_Pd,pixel_Fa
0,1.0,19.0,26.0,150.0,1376256.0,0.730769,0.000109
1,2.0,22.0,26.0,54.0,1376256.0,0.846154,0.000039
2,3.0,22.0,26.0,54.0,1376256.0,0.846154,0.000039
3,4.0,22.0,26.0,54.0,1376256.0,0.846154,0.000039
4,5.0,22.0,26.0,54.0,1376256.0,0.846154,0.000039
5,6.0,22.0,26.0,54.0,1376256.0,0.846154,0.000039
6,7.0,22.0,26.0,54.0,1376256.0,0.846154,0.000039
7,8.0,22.0,26.0,54.0,1376256.0,0.846154,0.000039
8,9.0,22.0,26.0,54.0,1376256.0,0.846154,0.000039
9,10.0,22.0,26.0,54.0,1376256.0,0.846154,0.000039


# PixelNormalizeIoU

In [11]:
from sosmetrics.metrics import PixelNormalizedIoU
Metric = PixelNormalizedIoU(
    conf_thr=0.5,
    # debug = True
    )
# For test multiple format of input.
gt_img_list = []
pred_img_list = [] 

gt_img_list_chw = []
pred_img_list_chw = []

gt_img_paths = []
pred_img_paths = []
for image_name in tbar:
    tbar.set_description(f"Reading image_name={image_name}")
    gt_image_path = os.path.join(gt_path, f"{image_name}.png")  # 假设所有图片都是PNG格式，或者你可以根据实际情况修改
    pred_image_path = os.path.join(preds_path, f"{image_name}.png") 

    gt_img = cv2.imread(gt_image_path)
    pred_img = cv2.imread(pred_image_path)

    # for test [chw, chw, ...] format
    gt_img_list_chw.append(gt_img.transpose(2,1,0))
    pred_img_list_chw.append(pred_img.transpose(2,1,0))

    # for test [hwc, hwc] format
    gt_img_list.append(gt_img)
    pred_img_list.append(pred_img)

    # for test [path, path] format

    gt_img_paths.append(gt_image_path)
    pred_img_paths.append(pred_image_path)

    # for test single img or img path
    Metric.update(gt_img, pred_img)
    Metric.update(gt_image_path, pred_image_path)

print("Simultaneous test single image path and single image, (TP, FN, FP) should be two times as big as the following test.")
Metric.get()
Metric.reset()


print("Test image of list [hwc, hwc, ...]")
Metric.update(gt_img_list, pred_img_list) 
Metric.get()
Metric.reset()

print("Test image_path of list, [img_path, img_path, ...]")
Metric.update(gt_img_paths, pred_img_paths)
Metric.get()
Metric.reset()

print("Test image of np.array, bhwc")
Metric.update(np.stack(gt_img_list), np.stack(pred_img_list)) 
Metric.get()
Metric.reset()


print("Test image of tensor, bchw")
Metric.update(torch.from_numpy(np.stack(gt_img_list_chw)), torch.from_numpy(np.stack(pred_img_list_chw))) 
Metric.get()
Metric.reset()

print("Test image of list,  [chw, chw, ...]")
Metric.update(gt_img_list_chw, pred_img_list_chw) 
Metric.get()


Simultaneous test single image path and single image, (TP, FN, FP) should be two times as big as the following test.
PixelNormalizedIoU.update() took 0.00s each time.
+----------+
| nIoU-0.5 |
+----------+
|  0.7178  |
+----------+
PixelNormalizedIoU(conf_thr=0.5)
Test image of list [hwc, hwc, ...]
PixelNormalizedIoU.update() took 0.01s each time.
+----------+
| nIoU-0.5 |
+----------+
|  0.7178  |
+----------+
PixelNormalizedIoU(conf_thr=0.5)
Test image_path of list, [img_path, img_path, ...]
PixelNormalizedIoU.update() took 0.02s each time.
+----------+
| nIoU-0.5 |
+----------+
|  0.7178  |
+----------+
PixelNormalizedIoU(conf_thr=0.5)
Test image of np.array, bhwc
PixelNormalizedIoU.update() took 0.02s each time.
+----------+
| nIoU-0.5 |
+----------+
|  0.7178  |
+----------+
PixelNormalizedIoU(conf_thr=0.5)
Test image of tensor, bchw
PixelNormalizedIoU.update() took 0.02s each time.
+----------+
| nIoU-0.5 |
+----------+
|  0.7178  |
+----------+
PixelNormalizedIoU(conf_thr=0.5)
T

0.7178133052159646

In [12]:
Metric.table

,nIoU
0,0.717813


## HybridNormalizedIoU

In [13]:
from sosmetrics.metrics import HybridNormalizedIoU
Metric = HybridNormalizedIoU(
    conf_thr=0.5,
    dis_thr = [1,10],
    match_alg='forloop',
    # debug = True
    )
# For test multiple format of input.
gt_img_list = []
pred_img_list = [] 

gt_img_list_chw = []
pred_img_list_chw = []

gt_img_paths = []
pred_img_paths = []
for image_name in tbar:
    tbar.set_description(f"Reading image_name={image_name}")
    gt_image_path = os.path.join(gt_path, f"{image_name}.png")  # 假设所有图片都是PNG格式，或者你可以根据实际情况修改
    pred_image_path = os.path.join(preds_path, f"{image_name}.png") 

    gt_img = cv2.imread(gt_image_path)
    pred_img = cv2.imread(pred_image_path)

    # for test [chw, chw, ...] format
    gt_img_list_chw.append(gt_img.transpose(2,1,0))
    pred_img_list_chw.append(pred_img.transpose(2,1,0))

    # for test [hwc, hwc] format
    gt_img_list.append(gt_img)
    pred_img_list.append(pred_img)

    # for test [path, path] format

    gt_img_paths.append(gt_image_path)
    pred_img_paths.append(pred_image_path)

    # for test single img or img path
    Metric.update(gt_img, pred_img)
    Metric.update(gt_image_path, pred_image_path)

print("Simultaneous test single image path and single image, (TP, FN, FP) should be two times as big as the following test.")
Metric.get()
Metric.reset()


print("Test image of list [hwc, hwc, ...]")
Metric.update(gt_img_list, pred_img_list) 
Metric.get()
Metric.reset()

print("Test image_path of list, [img_path, img_path, ...]")
Metric.update(gt_img_paths, pred_img_paths)
Metric.get()
Metric.reset()

print("Test image of np.array, bhwc")
Metric.update(np.stack(gt_img_list), np.stack(pred_img_list)) 
Metric.get()
Metric.reset()


print("Test image of tensor, bchw")
Metric.update(torch.from_numpy(np.stack(gt_img_list_chw)), torch.from_numpy(np.stack(pred_img_list_chw))) 
Metric.get()
Metric.reset()

print("Test image of list,  [chw, chw, ...]")
Metric.update(gt_img_list_chw, pred_img_list_chw) 
Metric.get()


Simultaneous test single image path and single image, (TP, FN, FP) should be two times as big as the following test.
HybridNormalizedIoU.update() took 0.00s each time.
+----------+--------+--------+--------+--------+--------+--------+--------+--------+--------+--------+
| Dis—Thr  |  1.0   |  2.0   |  3.0   |  4.0   |  5.0   |  6.0   |  7.0   |  8.0   |  9.0   |  10.0  |
+----------+--------+--------+--------+--------+--------+--------+--------+--------+--------+--------+
| nIoU-0.5 | 0.6161 | 0.6785 | 0.6785 | 0.6785 | 0.6785 | 0.6785 | 0.6785 | 0.6785 | 0.6785 | 0.6785 |
+----------+--------+--------+--------+--------+--------+--------+--------+--------+--------+--------+
HybridNormalizedIoU(conf_thr=0.5)
Test image of list [hwc, hwc, ...]
HybridNormalizedIoU.update() took 0.00s each time.
+----------+--------+--------+--------+--------+--------+--------+--------+--------+--------+--------+
| Dis—Thr  |  1.0   |  2.0   |  3.0   |  4.0   |  5.0   |  6.0   |  7.0   |  8.0   |  9.0   | 

array([0.61610628, 0.67848973, 0.67848973, 0.67848973, 0.67848973,
       0.67848973, 0.67848973, 0.67848973, 0.67848973, 0.67848973])

In [14]:
Metric.table

,0,1,2,3,4,5,6,7,8,9
Dis-Thr,1.000000,2.00000,3.00000,4.00000,5.00000,6.00000,7.00000,8.00000,9.00000,10.00000
nIoU,0.616106,0.67849,0.67849,0.67849,0.67849,0.67849,0.67849,0.67849,0.67849,0.67849


# Pixel ROC Precision Recall Metric

In [15]:
from sosmetrics.metrics import PixelROCPrecisionRecall
Metric = PixelROCPrecisionRecall(
    bins=30,
    # debug = True
    )
# For test multiple format of input.
gt_img_list = []
pred_img_list = [] 

gt_img_list_chw = []
pred_img_list_chw = []

gt_img_paths = []
pred_img_paths = []
for image_name in tbar:
    tbar.set_description(f"Reading image_name={image_name}")
    gt_image_path = os.path.join(gt_path, f"{image_name}.png")  # 假设所有图片都是PNG格式，或者你可以根据实际情况修改
    pred_image_path = os.path.join(preds_path, f"{image_name}.png") 

    gt_img = cv2.imread(gt_image_path)
    pred_img = cv2.imread(pred_image_path)

    # for test [chw, chw, ...] format
    gt_img_list_chw.append(gt_img.transpose(2,1,0))
    pred_img_list_chw.append(pred_img.transpose(2,1,0))

    # for test [hwc, hwc] format
    gt_img_list.append(gt_img)
    pred_img_list.append(pred_img)

    # for test [path, path] format

    gt_img_paths.append(gt_image_path)
    pred_img_paths.append(pred_image_path)

    # for test single img or img path
    Metric.update(gt_img, pred_img)
    Metric.update(gt_image_path, pred_image_path)

print("Simultaneous test single image path and single image, (TP, FN, FP) should be two times as big as the following test.")
Metric.get()
Metric.reset()


print("Test image of list [hwc, hwc, ...]")
Metric.update(gt_img_list, pred_img_list) 
Metric.get()
Metric.reset()

print("Test image_path of list, [img_path, img_path, ...]")
Metric.update(gt_img_paths, pred_img_paths)
Metric.get()
Metric.reset()

print("Test image of np.array, bhwc")
Metric.update(np.stack(gt_img_list), np.stack(pred_img_list)) 
Metric.get()
Metric.reset()


print("Test image of tensor, bchw")
Metric.update(torch.from_numpy(np.stack(gt_img_list_chw)), torch.from_numpy(np.stack(pred_img_list_chw))) 
Metric.get()
Metric.reset()

print("Test image of list,  [chw, chw, ...]")
Metric.update(gt_img_list_chw, pred_img_list_chw) 
Metric.get()


Simultaneous test single image path and single image, (TP, FN, FP) should be two times as big as the following test.
PixelROCPrecisionRecall.update() took 0.08s each time.
+--------------------+----------------------+-------------------------------------+
|      AUC_ROC       | AUC_PR(AUC function) | AP(BinaryAveragePrecision function) |
+--------------------+----------------------+-------------------------------------+
| 0.9427307844161987 |  0.8806460499763489  |              0.8124518              |
+--------------------+----------------------+-------------------------------------+
PixelROCPrecisionRecall
Test image of list [hwc, hwc, ...]
PixelROCPrecisionRecall.update() took 0.09s each time.
+--------------------+----------------------+-------------------------------------+
|      AUC_ROC       | AUC_PR(AUC function) | AP(BinaryAveragePrecision function) |
+--------------------+----------------------+-------------------------------------+
| 0.9427307844161987 |  0.8806460499763489

(0.9427307844161987,
 0.8806460499763489,
 array([9.45113607e-06, 6.25229004e-05, 6.90659945e-05, 7.19740347e-05,
        7.63361022e-05, 7.77901223e-05, 7.85171287e-05, 7.99711488e-05,
        8.14251762e-05, 8.36062027e-05, 8.57872365e-05, 8.72412566e-05,
        9.16033168e-05, 9.52383707e-05, 9.66923908e-05, 9.66923908e-05,
        9.74194045e-05, 9.81464109e-05, 9.96004383e-05, 1.00327445e-04,
        1.02508478e-04, 1.05416519e-04, 1.06870539e-04, 1.08324559e-04,
        1.10505593e-04, 1.13413633e-04, 1.16321673e-04, 1.25772814e-04,
        1.36677976e-04, 1.00000000e+00], dtype=float32),
 array([0.45789474, 0.79868424, 0.8144737 , 0.8210526 , 0.8276316 ,
        0.83157897, 0.8342105 , 0.8381579 , 0.8407895 , 0.8407895 ,
        0.84210527, 0.84605265, 0.84605265, 0.8473684 , 0.8486842 ,
        0.85      , 0.85263157, 0.85394734, 0.85394734, 0.8578947 ,
        0.8605263 , 0.8605263 , 0.8618421 , 0.86315787, 0.8644737 ,
        0.86973685, 0.86973685, 0.8723684 , 0.8855263 , 1

In [16]:
Metric.table

,AUC_ROC,AUC_PR,AP
0,0.942731,0.880646,0.812452


# Piexl Precision Recall F1 IoU 

In [17]:
from sosmetrics.metrics import PixelPrecisionRecallF1IoU
Metric = PixelPrecisionRecallF1IoU(
    conf_thr=0.5,
    # debug = True
    )
# For test multiple format of input.
gt_img_list = []
pred_img_list = [] 

gt_img_list_chw = []
pred_img_list_chw = []

gt_img_paths = []
pred_img_paths = []
for image_name in tbar:
    tbar.set_description(f"Reading image_name={image_name}")
    gt_image_path = os.path.join(gt_path, f"{image_name}.png")  # 假设所有图片都是PNG格式，或者你可以根据实际情况修改
    pred_image_path = os.path.join(preds_path, f"{image_name}.png") 

    gt_img = cv2.imread(gt_image_path)
    pred_img = cv2.imread(pred_image_path)

    # for test [chw, chw, ...] format
    gt_img_list_chw.append(gt_img.transpose(2,1,0))
    pred_img_list_chw.append(pred_img.transpose(2,1,0))

    # for test [hwc, hwc] format
    gt_img_list.append(gt_img)
    pred_img_list.append(pred_img)

    # for test [path, path] format

    gt_img_paths.append(gt_image_path)
    pred_img_paths.append(pred_image_path)

    # for test single img or img path
    Metric.update(gt_img, pred_img)
    Metric.update(gt_image_path, pred_image_path)

print("Simultaneous test single image path and single image, (TP, FN, FP) should be two times as big as the following test.")
Metric.get()
Metric.reset()


print("Test image of list [hwc, hwc, ...]")
Metric.update(gt_img_list, pred_img_list) 
Metric.get()
Metric.reset()

print("Test image_path of list, [img_path, img_path, ...]")
Metric.update(gt_img_paths, pred_img_paths)
Metric.get()
Metric.reset()

print("Test image of np.array, bhwc")
Metric.update(np.stack(gt_img_list), np.stack(pred_img_list)) 
Metric.get()
Metric.reset()


print("Test image of tensor, bchw")
Metric.update(torch.from_numpy(np.stack(gt_img_list_chw)), torch.from_numpy(np.stack(pred_img_list_chw))) 
Metric.get()
Metric.reset()

print("Test image of list,  [chw, chw, ...]")
Metric.update(gt_img_list_chw, pred_img_list_chw) 
Metric.get()


Simultaneous test single image path and single image, (TP, FN, FP) should be two times as big as the following test.
PixelPrecisionRecallF1IoU.update() took 0.01s each time.
+---------------+------------+--------+---------+
| Precision-0.5 | Recall-0.5 | F1-0.5 | IOU-0.5 |
+---------------+------------+--------+---------+
|     0.8293    |   0.8500   | 0.8395 | 0.72340 |
+---------------+------------+--------+---------+
PixelPrecisionRecallF1IoU
Test image of list [hwc, hwc, ...]
PixelPrecisionRecallF1IoU.update() took 0.02s each time.
+---------------+------------+--------+---------+
| Precision-0.5 | Recall-0.5 | F1-0.5 | IOU-0.5 |
+---------------+------------+--------+---------+
|     0.8293    |   0.8500   | 0.8395 | 0.72340 |
+---------------+------------+--------+---------+
PixelPrecisionRecallF1IoU
Test image_path of list, [img_path, img_path, ...]
PixelPrecisionRecallF1IoU.update() took 0.03s each time.
+---------------+------------+--------+---------+
| Precision-0.5 | Recall

(array([0.82926829]), array([0.85]), array([0.83950617]), array([0.72340426]))

In [18]:
Metric.table

,Precision-0.5,Recall-0.5,F1-0.5,IOU-0.5
0,0.829268,0.85,0.839506,0.723404


In [19]:
from sosmetrics.metrics import TargetPdPixelFaROC
Metric = TargetPdPixelFaROC(dis_thr=[1, 10],
    conf_thr=50,
    match_alg='forloop',
    )

gt_img_list = []
pred_img_list = [] 

gt_img_list_chw = []
pred_img_list_chw = []

gt_img_paths = []
pred_img_paths = []
for image_name in tbar:
    tbar.set_description(f"Reading image_name={image_name}")
    gt_image_path = os.path.join(gt_path, f"{image_name}.png")  # 假设所有图片都是PNG格式，或者你可以根据实际情况修改
    pred_image_path = os.path.join(preds_path, f"{image_name}.png") 

    gt_img = cv2.imread(gt_image_path)
    pred_img = cv2.imread(pred_image_path)

    # for test [chw, chw, ...] format
    gt_img_list_chw.append(gt_img.transpose(2,1,0))
    pred_img_list_chw.append(pred_img.transpose(2,1,0))

    # for test [hwc, hwc] format
    gt_img_list.append(gt_img)
    pred_img_list.append(pred_img)

    # for test [path, path] format

    gt_img_paths.append(gt_image_path)
    pred_img_paths.append(pred_image_path)

    # for test single img or img path
    Metric.update(gt_img, pred_img)
    Metric.update(gt_image_path, pred_image_path)

print("Simultaneous test single image path and single image, (TP, FN, FP) should be two times as big as the following test.")
Metric.get()
Metric.reset()


print("Test image of list [hwc, hwc, ...]")
Metric.update(gt_img_list, pred_img_list) 
Metric.get()
Metric.reset()

print("Test image_path of list, [img_path, img_path, ...]")
Metric.update(gt_img_paths, pred_img_paths)
Metric.get()
Metric.reset()

print("Test image of np.array, bhwc")
Metric.update(np.stack(gt_img_list), np.stack(pred_img_list)) 
Metric.get()
Metric.reset()


print("Test image of tensor, bchw")
Metric.update(torch.from_numpy(np.stack(gt_img_list_chw)), torch.from_numpy(np.stack(pred_img_list_chw))) 
Metric.get()
Metric.reset()

print("Test image of list,  [chw, chw, ...]")
Metric.update(gt_img_list_chw, pred_img_list_chw) 
Metric.get()


Simultaneous test single image path and single image, (TP, FN, FP) should be two times as big as the following test.
TargetPdPixelFaROC.update() took 0.12s each time.
+---------+--------+--------+--------+--------+--------+--------+--------+--------+--------+--------+
| Dis—Thr |  1.0   |  2.0   |  3.0   |  4.0   |  5.0   |  6.0   |  7.0   |  8.0   |  9.0   |  10.0  |
+---------+--------+--------+--------+--------+--------+--------+--------+--------+--------+--------+
| AUC-ROC | 0.3462 | 0.3654 | 0.4039 | 0.4231 | 0.4231 | 0.4231 | 0.4231 | 0.4231 | 0.4039 | 0.4039 |
+---------+--------+--------+--------+--------+--------+--------+--------+--------+--------+--------+
TargetPdPixelFaROC(match_alg=forloop)
Test image of list [hwc, hwc, ...]
TargetPdPixelFaROC.update() took 0.18s each time.
+---------+--------+--------+--------+--------+--------+--------+--------+--------+--------+--------+
| Dis—Thr |  1.0   |  2.0   |  3.0   |  4.0   |  5.0   |  6.0   |  7.0   |  8.0   |  9.0   |  10.0

(array([[0.        , 0.80769231, 0.80769231, 0.80769231, 0.80769231,
         0.80769231, 0.80769231, 0.80769231, 0.80769231, 0.80769231,
         0.80769231, 0.80769231, 0.80769231, 0.80769231, 0.80769231,
         0.80769231, 0.80769231, 0.80769231, 0.80769231, 0.80769231,
         0.80769231, 0.80769231, 0.80769231, 0.76923077, 0.73076923,
         0.73076923, 0.73076923, 0.73076923, 0.73076923, 0.73076923,
         0.73076923, 0.73076923, 0.73076923, 0.69230769, 0.69230769,
         0.69230769, 0.69230769, 0.69230769, 0.69230769, 0.65384615,
         0.65384615, 0.65384615, 0.65384615, 0.65384615, 0.65384615,
         0.65384615, 0.65384615, 0.65384615, 0.69230769, 0.61538462],
        [0.        , 0.84615385, 0.84615385, 0.84615385, 0.84615385,
         0.84615385, 0.84615385, 0.84615385, 0.84615385, 0.84615385,
         0.84615385, 0.84615385, 0.84615385, 0.84615385, 0.84615385,
         0.84615385, 0.84615385, 0.84615385, 0.84615385, 0.84615385,
         0.84615385, 0.84615385, 